# Access Db2 Warehouse on Cloud and Db2 with Python

* [Db2 Warehouse on Cloud](https://console.bluemix.net/catalog/services/db2-warehouse) 서비스를 활용하기 위해서 Python jdbc 라이브러리를 활용합니다. 
* 수행전에 Db2 Warehouse on cloud 서비스를 생성하고, 접속 정보를 가져와야 합니다.

*** "Db2WH connection info" [github](https://github.com/moreal70/IBM-Watson-Studio/blob/master/Db2WH-connection%20info.ipynb) 참고하세요 ***  


## Table of contents

1. [Setup](#Setup) 
1. [Import the *ibmdbpy* Python library](#Import-the-ibmdbpy-Python-library)
1. [Identify and enter the database connection credentials](#Identify-and-enter-the-database-connection-credentials)
1. [Create the database connection](#Create-the-database-connection)
1. [Use dataframe to read and manipulate tables](#Use-dataframe-to-read-and-manipulate-tables)
1. [Close the database connection](#Close-the-database-connection)
1. [Summary](#Summary)


## Setup

Before you begin you will need **Db2 Warehouse on Cloud** which is a fully-managed, enterprise-class, cloud data warehouse service, purpose-built for analytics. It offers massively parallel processing (MPP) scale and compatibility with a wide range of business intelligence (BI) tools.  


## Import the *ibmdbpy* Python library

Python support for Db2 Warehouse on Cloud and Db2 is provided by the <a href="https://pypi.python.org/pypi/ibmdbpy" target="_blank" rel="noopener noreferrer">ibmdbpy Python library</a>. Connecting to Db2 Warehouse on Cloud or Db2 is also enabled by a Db2 driver, libdb2.so.

The JDBC Connection is based on a Java virtual machine. From the ibmdbpy library you can use JDBC to connect to a remote Db2 Warehouse on Cloud/Db2 instance. To be able to use JDBC to connect, we need to import the *JayDeBeApi* package.

* 해당 environment 시스템에 모듈이 설치가 되지 않았을 경우, 아래 cell 을 실행해서 설치합니다. 

In [1]:
!pip install jaydebeapi --user  
!pip install ibmdbpy --user 

Requirement not upgraded as not directly required: jaydebeapi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: JPype1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jaydebeapi)
Requirement not upgraded as not directly required: ibmdbpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lazy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibmdbpy)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibmdbpy)
Requirement not upgraded as not directly required: pypyodbc in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibmdbpy)
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibmdbpy)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/li

In [4]:
import jaydebeapi
from ibmdbpy import IdaDataBase
from ibmdbpy import IdaDataFrame

In [5]:
import os
#os.environ['CLASSPATH'] = "/usr/local/src/data-connectors-1.4.1/db2jcc4-10.5.0.6.jar"
# for Watson Studio Environments please use the following location
os.environ['CLASSPATH'] = "/opt/ibm/jdbc/db2jcc4.jar"


In [6]:
import jpype
args='-Djava.class.path=%s' % os.environ['CLASSPATH']
jvm = jpype.getDefaultJVMPath()

***실제로 jar 화일이 존재하는 지 확인해봤습니다.***

In [7]:
!ls -al /opt/ibm/jdbc/db2jcc4.jar
!ls -al /opt/ibm/java-x86_64-80/jre/bin/classic/libjvm.so
args

-r--r--r-- 1 root root 3924511 Mar 15 13:16 /opt/ibm/jdbc/db2jcc4.jar
-rwxr-xr-x 1 root root 115232 Feb 14 18:38 /opt/ibm/java-x86_64-80/jre/bin/classic/libjvm.so


'-Djava.class.path=/opt/ibm/jdbc/db2jcc4.jar'

In [8]:
jpype.startJVM(jvm, args)


## Identify and enter the database connection credentials

Connecting to Db2 Warehouse on Cloud or a Db2 database requires the following information:
* Database name 
* Host DNS name or IP address 
* Host port
* Connection protocol
* User ID
* User password

All of this information must be captured in a connection string in a subsequent step. Provide the Db2 Warehouse on Cloud or Db2 connection information as shown:

In [7]:
dsn_uid = "dash33218";  # e.g.  db104434
dsn_pwd = "6__r9ltKnMVW"   # e.g. xxxx
dsn_hostname ="dashdb-entry-yp-syd01-01.services.au-syd.bluemix.net"  # e.g.  awh-yp-small03.services.dal.bluemix.net
dsn_port = "50000"   # e.g.  50001
dsn_database = "BLUDB"   # e.g. BLUDB 

## Create the database connection

The following code snippet creates a connection string `connection_string`
and uses the `connection_string` to create a Db2 connection object:


In [8]:
connection_string='jdbc:db2://'+dsn_hostname+':'+dsn_port+'/'+dsn_database+':user='+dsn_uid+';password='+dsn_pwd+";" 
idadb=IdaDataBase(dsn=connection_string)

## Use dataframe to read and manipulate tables

You can now use the connection object `conn` to query the database:

In [9]:
df=idadb.show_tables(show_all = True)
df.head(5)

,TABSCHEMA,TABNAME,OWNER,TYPE
0,GOSALES,BRANCH,DB2INST1,T
1,GOSALES,CONVERSION_RATE,DB2INST1,T
2,GOSALES,COUNTRY,DB2INST1,T
3,GOSALES,CURRENCY_LOOKUP,DB2INST1,T
4,GOSALES,EURO_CONVERSION,DB2INST1,T


In [10]:
idadb.exists_table_or_view('GOSALESDW.EMP_EXPENSE_FACT')

True

Using our previously opened IdaDataBase instance named ‘idadb’, we can open one or several IdaDataFrame objects. They behave like pointers to remote tables.

Let us open the *EMP_EXPENSE_FACT* data set, assuming it is stored in the database under the name ‘GOSALESDW.EMP_EXPENSE_FACT’. The following cell assigns the dataset to a pandas DataFrame.

The [Pandas data analysis library](http://pandas.pydata.org/) provides high-performance, easy-to-use data structures and data analysis tools for the Python programming language. Pandas allows easy processing and manipulation of tabular data, so it is a perfect fit for data extracted from relational databases.


In [11]:
idadf = IdaDataFrame(idadb, 'GOSALESDW.EMP_EXPENSE_FACT')

You can very easily explore the data in the IdaDataFrame by using built in functions.

Use IdaDataFrame.head to get the first n records of your data set (default 5):

In [12]:
idadf.head(5)

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
0,20100131,11177,43613,4770,2124,8056,0.03,59.95
1,20100131,11177,43613,4770,2103,8050,15.00,181.66
2,20100131,11177,43613,4770,2101,8049,150.00,1816.56
3,20100131,11177,43613,4770,2122,8054,0.11,219.80
4,20100131,11177,43613,4770,2120,8052,0.08,159.86


Use IdaDataFrame.tail to get the last n records of your data set (default 5):

In [13]:
idadf.tail(5)

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
127979,20130731,11125,43633,4207,2104,8050,11.25,216.35
127980,20130731,11125,43633,4207,2114,8050,1.75,49.88
127981,20130731,11125,43633,4207,2120,8052,0.08,257.84
127982,20130731,11125,43633,4207,2122,8054,0.11,354.53
127983,20130731,11125,43633,4207,2124,8056,0.03,96.69


__Note__: Because Db2 Warehouse on Cloud operates on a distributed system, the order of rows using IdaDataFrame.head and IdaDataFrame.tail is not guaranteed unless the table is sorted (using an ‘ORDER BY’ clause) or a column is declared as index for the IdaDataFrame (parameter/attribute indexer).

IdaDataFrame also implements most attributes that are available in a pandas DataFrame:


In [14]:
idadf.shape

(127984, 8)

In [15]:
idadf.columns

Index(['DAY_KEY', 'ORGANIZATION_KEY', 'POSITION_KEY', 'EMPLOYEE_KEY',
       'EXPENSE_TYPE_KEY', 'ACCOUNT_KEY', 'EXPENSE_UNIT_QUANTITY',
       'EXPENSE_TOTAL'],
      dtype='object')

Several standard statistics functions from the pandas interface are also available for IdaDataFrame. For example, let us calculate the covariance matrix for the iris data set:

In [16]:
idadf.cov()

,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
DAY_KEY,1.074445e+08,-1301.774305,-2699.336397,-74463.200864,-2541.104007,-88.733494,-2747.250164,3.387493e+05
ORGANIZATION_KEY,-1.301774e+03,977.978493,-60.746262,2228.417559,-27.240468,0.756326,11.186590,-2.999219e+03
POSITION_KEY,-2.699336e+03,-60.746262,148.234472,-2070.934630,10.284910,-1.006254,-13.697657,1.101108e+03
EMPLOYEE_KEY,-7.446320e+04,2228.417559,-2070.934630,89393.601947,-237.530049,39.144365,525.387975,4.739903e+04
EXPENSE_TYPE_KEY,-2.541104e+03,-27.240468,10.284910,-237.530049,88.103306,4.663223,26.490807,5.577918e+03
ACCOUNT_KEY,-8.873349e+01,0.756326,-1.006254,39.144365,4.663223,6.414971,-92.920363,-2.669485e+03
EXPENSE_UNIT_QUANTITY,-2.747250e+03,11.186590,-13.697657,525.387975,26.490807,-92.920363,3331.325768,7.674054e+04
EXPENSE_TOTAL,3.387493e+05,-2999.218552,1101.107528,47399.031411,5577.918013,-2669.484571,76740.540006,4.321078e+06


It is possible to subset the rows of an IdaDataFrame by accessing the IdaDataFrame with a slice object. You can also use the IdaDataFrame.loc attribute, which contains an ibmdbpy.Loc object. However, the row selection might be inaccurate if the current IdaDataFrame is not sorted or does not contain an indexer. This is due to the fact that Db2 Warehouse on Cloud stores the data across several nodes if available. Moreover, because Db2 Warehouse on Cloud is a column oriented database, row numbers are undefined:

In [17]:
idadf_new = idadf[0:9] # Select the first 10 rows
idadf_new.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ibmdbpy/indexing.py:118: UserWarning: Row order is not guaranteed if no indexer was given and the dataset was not sorted
  " was given and the dataset was not sorted")


,DAY_KEY,ORGANIZATION_KEY,POSITION_KEY,EMPLOYEE_KEY,EXPENSE_TYPE_KEY,ACCOUNT_KEY,EXPENSE_UNIT_QUANTITY,EXPENSE_TOTAL
0,20110228,11118,43627,4802,2122,8054,0.11,261.73
1,20110228,11118,43627,4802,2124,8056,0.03,71.38
2,20110331,11118,43627,4802,2101,8049,172.50,2619.82
3,20110331,11118,43627,4802,2120,8052,0.08,223.99
4,20110331,11118,43627,4802,2114,8050,8.00,180.00


## Close the database connection

To ensure expected behaviors, IdaDataBase instances need to be closed. Closing the *IdaDataBase* is equivalent to closing the connection: once the connection is closed, it is no longer possible to use the *IdaDataBase* instance and any IdaDataFrame instances that were opened on this connection.

In [18]:
idadb.close()

Connection closed.


## Summary

This notebook demonstrated how to establish a connection to a Db2 Warehouse on Cloud / Db2 database from Python using the ibmdbpy library.

### Authors

**Saeed Aghabozorgi**, PhD, is a Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in the data mining field and an expert in developing advanced analytic methods like machine learning and statistical modelling on large data sets.

**Polong Lin** is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds an M.Sc. in Cognitive Psychology.

Copyright © 2016, 2018 Cognitive Class. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/" rel="noopener noreferrer" target="_blank">MIT License</a>.